In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import RidgeCV, LinearRegression
from sklearn import metrics
import sys
import copy
import statsmodels.api as sm
pd.options.mode.chained_assignment = None
from statsmodels.stats.outliers_influence import variance_inflation_factor
pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 2000
%matplotlib inline

dfTemp=pd.read_csv(r'C:\Users\Jaesung Park\Desktop\FP\GroupProject\kc_house_data.csv',keep_default_na=False)

In [2]:
data = dfTemp.reindex(np.random.permutation(dfTemp.index))
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('id')
data.price = data.price.astype(int)
data.bathrooms = data.bathrooms.astype(int)
data.floors = data.floors.astype(int)
data["house_age"] = data["date"].dt.year - data['yr_built']
data['renovated'] = data['yr_renovated'].apply(lambda yr: 0 if yr == 0 else 1)

data=data.drop('date', axis=1)
data=data.drop('yr_renovated', axis=1)
data=data.drop('yr_built', axis=1)

dataset = data[data.columns.difference(['id', 'long'])]
#X = dataset[['house_age','sqft_living','grade','condition', 
#'sqft_living15','bathrooms','view','sqft_basement','floors','waterfront',
#'renovated','bedrooms']]


In [3]:
fall = [] # 1빼고 다 넣은 리스트
for i in range(len(dataset.columns)):
    if dataset.columns[i] in ([]):
        fall.append(dataset.columns[i])

five = [] # 1빼고 다 넣은 리스트
for i in range(len(dataset.columns)):
    if dataset.columns[i] not in (['price', 'sqft_living']):
        five.append(dataset.columns[i])
        
fifthteen = [] # 2빼고 다 넣은 리스트
for i in range(len(dataset.columns)):
    if dataset.columns[i] not in (['price', 'sqft_living','grade']):
        fifthteen.append(dataset.columns[i])

thirty = [] # 4빼고 다 넣은 리스트
for i in range(len(dataset.columns)):
    if dataset.columns[i] not in (['price', 'sqft_living','sqft_above','grade','sqft_living15']):
        thirty.append(dataset.columns[i])    

half = [] # 반타작
for i in range(len(dataset.columns)):
    if dataset.columns[i] not in (['price','sqft_living','sqft_above','grade','sqft_living15',
       'bathrooms', 'view']):
        half.append(dataset.columns[i])
        
seventy = [] # 8 in
for i in range(len(dataset.columns)):
    if dataset.columns[i] in (['condition', 'house_age','zipcode', 'sqft_lot15']):
        seventy.append(dataset.columns[i]) 
        
eightyfive = [] # 10 in
for i in range(len(dataset.columns)):
    if dataset.columns[i] in (['condition', 'house_age']):
        eightyfive.append(dataset.columns[i]) 
        
ninetyfive = [] # 11 in
for i in range(len(dataset.columns)):
    if dataset.columns[i] in (['condition']):
        ninetyfive.append(dataset.columns[i]) 


In [4]:
testing = dataset
#alphas = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1,0.5, 1, 10, 15, 20, 30, 40, 100]
#regressor=RidgeCV(alphas=alphas, store_cv_values=True)
regressor = LinearRegression(fit_intercept=True)

In [5]:
## exclude the lefts
rounded = round(testing.shape[0]/10)*10
left = testing.shape[0] - rounded
testing2 = testing.iloc[left:]

k = testing2.shape[0]
fold = 10
point = round(k/fold)
mylist = []

start = 0
end= point

# separating dataset into # number of folds and put each fold into list
for i in range(fold):
    x = testing2.iloc[start:end]
    start = start + point
    end = end + point
    mylist.append(x)

In [6]:
RidgeCV_MAE = []
RidgeCV_MSE = []
RidgeCV_RMSE = []
alphasL = []
r2_list = []
predicted = []
temp = copy.deepcopy(mylist)

In [7]:
hm = ninetyfive
for j in range(fold):
    mylist = copy.deepcopy(temp)
    testing_set = mylist[j]  
    appended_data = []
    for z in range(fold):
        if not z == j:
            appended_data.append(mylist[z])
    training_set = pd.concat(appended_data, ignore_index = True)
     
    #Regression imputation
    for p in range(len(testing_set.columns)):
        if testing_set.columns[p] in hm :
            hm.append('price')
            kfold_testing_X =  testing_set[testing.columns.difference(hm)]
            kfold_testing_Y =  testing_set[testing_set.columns[p]]
            kfold_training_X = training_set[testing.columns.difference(hm)]
            kfold_training_Y = training_set[testing_set.columns[p]] 
            regressor.fit(kfold_training_X, kfold_training_Y)
            kfold_pred = regressor.predict(kfold_testing_X)   
            testing_set[testing_set.columns[p]] = kfold_pred
            hm.remove('price')
            
    testing_set_X = testing_set[testing.columns.difference(['price'])]
    testing_set_Y = np.log(testing_set['price'])
    training_set_X = training_set[testing.columns.difference(['price'])]
    training_set_Y = np.log(training_set['price'])

    
    regressor.fit(training_set_X, training_set_Y)
    y_pred = regressor.predict(testing_set_X)
    MAE = metrics.mean_absolute_error(testing_set_Y, y_pred)
    MSE = metrics.mean_squared_error(testing_set_Y, y_pred)
    RMSE = np.sqrt(metrics.mean_squared_error(testing_set_Y, y_pred))
    r2 = metrics.r2_score(testing_set_Y, y_pred)
    py = y_pred
    
    RidgeCV_MAE.append(MAE)
    RidgeCV_MSE.append(MSE)
    RidgeCV_RMSE.append(RMSE)
    r2_list.append(r2)
    predicted.append(py)


In [8]:
np.mean(RidgeCV_MAE)

0.1986365812247166

In [9]:
np.mean(RidgeCV_MSE)

0.06625177155502586

In [10]:
np.mean(RidgeCV_RMSE)

0.25735667830567405

In [11]:
np.mean(r2_list)

0.7609207751349775